In [1]:
import numpy as np #numpy dipake kalau di program ada kebutuhan array
import pandas as pd #pandas dipake kalau di program ada kebutuhan nampilin data frame
import nltk #natural language toolkit dipakai kalau diprogram ada kebutuhan manipulasi atau edit kata-kata
import sklearn #scikit learn dipakai kalau mau melakukan ml di program
import joblib #library buat nyimpen model ml buat dipakai nanti 
from matplotlib import pyplot #kalau di program mau membuat tampilan kayak MATLAB dengan x dan y
import matplotlib.pyplot as plt
from nltk.corpus import stopwords #untuk mendapatkan list stopwords yang terdaftar pada library nltk.corpus
from sklearn.feature_extraction.text import TfidfVectorizer #tf-idf vectorizer ini dipakai untuk merubah kata pada data menjadi numerik agar bisa dibaca komputer
from sklearn import metrics #modul yang menilai prediksi error, intinya kalau mau make ml di program setidaknya pakai metrics, disini prediksi errornya pakai roc auc
from sklearn.metrics import roc_curve, roc_auc_score
import joblib as jb
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split 
from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder #buat encode label yang dpilih jadi nilai int
from sklearn.ensemble import RandomForestClassifier

In [2]:
data = pd.read_excel('data_training.xlsx')

In [3]:
data.head(5)

,Nama,Judul,Subject
0,"AGUSTIN MAULIYA SAFITRI , 1417021005",STUDI PENINGKATAN IMUNOSTIMULAN DARI KOMBINASI...,Q SCIENCE
1,"SAIFUL ANWAR , 1417051129",PENERAPAN DJANGO REST FRAMEWORK DAN TEKNOLOGI_...,Q SCIENCE
2,"AFRISA HERNI PUTRI , (1217021002)",UJI EFEKTIVITAS EKSTRAK RIMPANG RUMPUT TEKI (C...,Q SCIENCE
3,"AGNES MALUDFI PUTRI, 1217031004",ANALISIS TEORI ANTRIAN MULTI-CHANNEL DENGAN DI...,Q SCIENCE
4,"ALEXANDER DICKY K. N , 1218011008",PERBANDINGAN EFEK PEMBERIAN EKSTRAK TEMULAWAK_...,Q SCIENCE


In [4]:
print (len(data['Judul']))

17180


In [5]:
#ngasih tahu kalau df subject itu unik 
data['Subject'].unique()

array(['Q SCIENCE', 'TEKNOLOGI', 'PERTANIAN', 'PENGOBATAN',
       'LANGUAGE AND LITERATURE', 'FINE ARTS', 'MUSIC AND BOOKS ON MUSIC',
       'L EDUCATION', 'POLITICAL SCIENCE'], dtype=object)

In [6]:
le = LabelEncoder()#manggil fungsinya
le.fit(list(data['Subject'].values))#ngambil value kolom subject 
data['Subject'] = le.transform(list(data['Subject']))#ngerubah isi kolom subject ke int

In [7]:
data['Subject'].unique()

array([7, 8, 5, 4, 2, 0, 3, 1, 6])

In [3]:
data['Judul']=data['Judul'].apply(lambda i:i.lower())
data['Judul']

0        studi peningkatan imunostimulan dari kombinasi...
1        penerapan django rest framework dan teknologi_...
2        uji efektivitas ekstrak rimpang rumput teki (c...
3        analisis teori antrian multi-channel dengan di...
4        perbandingan efek pemberian ekstrak temulawak_...
                               ...                        
17175    asean tourism strategic plan (atsp) 2016-2025 ...
17176    kepentingan australia mengaktifkan kembali per...
17177    kebijakan luar negeri indonesia dalam mengatas...
17178    globalisasi nilai-nilai posfeminisme dan_x000d...
17179    respon meksiko dalam menghadapi perubahan ling...
Name: Judul, Length: 17180, dtype: object

In [4]:
data['Judul'].str.split() #tempat nyeplit

0        [studi, peningkatan, imunostimulan, dari, komb...
1        [penerapan, django, rest, framework, dan, tekn...
2        [uji, efektivitas, ekstrak, rimpang, rumput, t...
3        [analisis, teori, antrian, multi-channel, deng...
4        [perbandingan, efek, pemberian, ekstrak, temul...
                               ...                        
17175    [asean, tourism, strategic, plan, (atsp), 2016...
17176    [kepentingan, australia, mengaktifkan, kembali...
17177    [kebijakan, luar, negeri, indonesia, dalam, me...
17178    [globalisasi, nilai-nilai, posfeminisme, dan_x...
17179    [respon, meksiko, dalam, menghadapi, perubahan...
Name: Judul, Length: 17180, dtype: object

In [5]:
token=data['Judul'].str.split() #deklarasiin tempat hasil split jadi token

In [6]:
#tempat ngasih tahu kalau ini stopwords-nya
stop = list(stopwords.words('indonesian'))
print(len(stop))
stop[:5]

758


['ada', 'adalah', 'adanya', 'adapun', 'agak']

In [7]:
def rubah(data):
    token=data.split()#nunjukin data pada df pas displit
    a = ' '.join([word for word in token if (word  not in stop)&(word.isalpha())])#isalpha supaya gak ribet jadi satu df aplhabet semua yang dipake
    return a

In [8]:
t = data['Judul'].apply(rubah)
#print(type(t))
t.loc[2]

'uji efektivitas ekstrak rimpang rumput teki rotundus obat imodium antidiare mencit musculus jantan diinduksi oleum'

In [1]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(data['Judul'])#merubah text jadi numerik dari df judul
print(vectorizer.vocabulary_)#untuk menampilkan hasil dari transform dalam bentuk dictionary
print(type(vectors))
vectors.shape

NameError: name 'TfidfVectorizer' is not defined

In [133]:
X_train, X_test, y_train, y_test = train_test_split(vectors, data['Subject'], test_size=0.2)

In [95]:
print (X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(13744, 18902) (13744,) (3436, 18902) (3436,)


## Logistic Regression

In [134]:
clf = LogisticRegression(solver='lbfgs', max_iter=1000,random_state=0)#lbfgs = Limited-memory Broyden–Fletcher–Goldfarb–Shanno Algorithm. iter(iteration) defaultnya 100, random_state defaultnya none = 0
clf.fit(X_train, y_train) #clf clasification
pred = clf.predict(X_test)
print (metrics.f1_score(y_test, pred, average='macro'))
print (metrics.accuracy_score(y_test, pred))

0.4870044991601596
0.719441210710128


In [135]:
text = ['EFEKTIVITAS PROGRAM BANDAR LAMPUNG MAKMUR DAN BERKEADILAN BAZNAS DALAM MENINGKATKAN KESEJAHTERAAN MASYARAKAT DI KOTA BANDAR LAMPUNG STUDY KASUS PADA KOTA BANDAR LAMPUNG']
s = (vectorizer.transform(text))
print (s.shape)
d = (clf.predict(s))
print (d)

(1, 18902)
[6]


In [137]:
le.inverse_transform(d)[0]

'POLITICAL SCIENCE'

# Gaussian NB

In [36]:
clf = MultinomialNB(alpha=1.0) #default alpha 1.0
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print (metrics.f1_score(y_test, pred, average='macro')) #f1 untuk ngukur antara precision dan recall, keseimbangan antara keduanya defaultnya 1 sama kayak akurasi
#average macro ini karena macro mengukur tiap label, kebetulan setiap judul yang dipakai dan ini make score f1 makanya macro cocok, bisa make weighted tapi f1 jadi gak guna
print (metrics.accuracy_score(y_test, pred))

0.40899597333102133
0.7255529685681025


In [34]:
text = ['EFEKTIVITAS PROGRAM BANDAR LAMPUNG MAKMUR DAN BERKEADILAN BAZNAS DALAM MENINGKATKAN KESEJAHTERAAN MASYARAKAT DI KOTA BANDAR LAMPUNG STUDY KASUS PADA KOTA BANDAR LAMPUNG']
#text[0] = text[0].lower()
s = (vectorizer.transform(text))
print (s.shape)
d = (clf.predict(s))
print (d)

(1, 18902)
[1]


In [35]:
le.inverse_transform(d)[0]

'L EDUCATION'

# Random Forest

In [23]:
clf = RandomForestClassifier(max_depth=None, random_state=0)
clf.fit(X_train, y_train) #clf clasification
pred = clf.predict(X_test)
print (metrics.f1_score(y_test, pred, average='macro'))
print (metrics.accuracy_score(y_test, pred))

0.4445927169173269
0.6507566938300349


In [24]:
text = ['EFEKTIVITAS PROGRAM BANDAR LAMPUNG MAKMUR DAN BERKEADILAN BAZNAS DALAM MENINGKATKAN KESEJAHTERAAN MASYARAKAT DI KOTA BANDAR LAMPUNG STUDY KASUS PADA KOTA BANDAR LAMPUNG']
#text[0] = text[0].lower()
s = (vectorizer.transform(text))
print (s.shape)
d = (clf.predict(s))
print (d)

(1, 18902)
[1]


In [25]:
le.inverse_transform(d)[0]

'L EDUCATION'

# ROC AUC

In [26]:
clf = LogisticRegression(solver='lbfgs', max_iter=1000).fit(X_train, y_train)
preds = clf.predict_proba(X_test)
roc_auc = roc_auc_score(y_test,preds, multi_class='ovr')
print(roc_auc)

0.9383188845602315


In [27]:
clf = RandomForestClassifier(max_depth=None, random_state=0).fit(X_train, y_train)
preds = clf.predict_proba(X_test)
roc_auc = roc_auc_score(y_test,preds, multi_class='ovr')
print(roc_auc)

0.8895450308193994


In [28]:
clf = MultinomialNB(alpha=1.0).fit(X_train, y_train)
preds = clf.predict_proba(X_test)
roc_auc = roc_auc_score(y_test,preds, multi_class='ovr')
print(roc_auc)

0.8588607906717729


# Saving model

In [29]:
filename = 'best.pkl'
joblib.dump(clf, filename)
print ("Model saved")

Model saved


In [30]:
clf = joblib.load('best.pkl')

In [31]:
clf

MultinomialNB()

In [32]:
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test, y_test)
print(result)

0.7255529685681025
